In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({
    'cont_9': np.random.rand(10)*100,
    'cat_0': ['dog'] * 5 + ['cat'] * 5,
    'cat_1': ['wolf'] * 9 + ['tiger'] * 1,
    'y': [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
})

In [3]:
df

,cont_9,cat_0,cat_1,y
0,80.644252,dog,wolf,1
1,70.533565,dog,wolf,0
2,72.630773,dog,wolf,1
3,4.394533,dog,wolf,1
4,65.162702,dog,wolf,1
5,12.661812,cat,wolf,1
6,92.999372,cat,wolf,0
7,8.023875,cat,wolf,0
8,31.829332,cat,wolf,0
9,2.913117,cat,tiger,0


In [4]:
means0 = df.groupby('cat_0')['y'].mean().to_dict()
means0

{'cat': 0.2, 'dog': 0.8}

In [5]:
means1 = df.groupby('cat_1')['y'].mean().to_dict()
means1

{'tiger': 0.0, 'wolf': 0.5555555555555556}

In [8]:
# Source: https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
def calc_smooth_mean(df1, df2, cat_name, target, weight):
    # Compute the global mean
    mean = df[target].mean()

    # Compute the number of values and the mean of each group
    agg = df.groupby(cat_name)[target].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + weight * mean) / (counts + weight)

    # Replace each value by the according smoothed mean
    if df2 is None:
        return df1[cat_name].map(smooth)
    else:
        return df1[cat_name].map(smooth),df2[cat_name].map(smooth.to_dict())

In [13]:
WEIGHT = 5
df['cat_0_enc'] = calc_smooth_mean(df1=df, df2=None, cat_name='cat_0', target='y', weight=WEIGHT)
df['cat_1_enc'] = calc_smooth_mean(df1=df, df2=None, cat_name='cat_1', target='y', weight=WEIGHT)

In [12]:
df

,cont_9,cat_0,cat_1,y,cat_0_enc,cat_1_enc
0,80.644252,dog,wolf,1,0.6,0.526316
1,70.533565,dog,wolf,0,0.6,0.526316
2,72.630773,dog,wolf,1,0.6,0.526316
3,4.394533,dog,wolf,1,0.6,0.526316
4,65.162702,dog,wolf,1,0.6,0.526316
5,12.661812,cat,wolf,1,0.4,0.526316
6,92.999372,cat,wolf,0,0.4,0.526316
7,8.023875,cat,wolf,0,0.4,0.526316
8,31.829332,cat,wolf,0,0.4,0.526316
9,2.913117,cat,tiger,0,0.4,0.454545


In [6]:
df['y'].mean()

0.5